In [1]:
import os
from os.path import join as pj
import argparse
import sys
sys.path.append("modules")
import utils
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, f1_score, roc_auc_score
import seaborn as sns
from joblib import Parallel, delayed
import rpy2.robjects as robjects
import umap
import re
import matplotlib.pyplot as plt
from sklearn import metrics
import scanpy as sc
import pandas as pd
import csv

2024-01-07 09:21:40.269074: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [2]:
parser = argparse.ArgumentParser()
parser.add_argument('--task', type=str, default='new_query_cl')
parser.add_argument('--reference', type=str, default='atlas_new_no_neap')
parser.add_argument('--experiment', type=str, default='c_3')
parser.add_argument('--model', type=str, default='default')
parser.add_argument('--init_model', type=str, default='')
parser.add_argument('--init_model_ref', type=str, default='sp_latest')
parser.add_argument('--method', type=str, default='midas_embed')
o, _ = parser.parse_known_args()  # for python interactive
# o = parser.parse_args()

In [3]:
# load latent variables
data_config = utils.gen_data_config(o.task)
data_config_ref = utils.gen_data_config(o.reference)
data_config_ref["raw_data_dirs"] += data_config["raw_data_dirs"]
data_config_ref["raw_data_frags"] += data_config["raw_data_frags"]
data_config_ref["combs"] = data_config["combs"]
data_config_ref["comb_ratios"] = data_config["comb_ratios"]
data_config_ref["s_joint"] = data_config["s_joint"]
for k, v in data_config_ref.items():
    vars(o)[k] = v
model_config = utils.load_toml("configs/model.toml")["default"]
if o.model != "default":
    model_config.update(utils.load_toml("configs/model.toml")[o.model])
for k, v in model_config.items():
    vars(o)[k] = v
o.s_joint, o.combs, *_ = utils.gen_all_batch_ids(o.s_joint, o.combs)

o.pred_dir = pj("result", o.task, o.experiment, o.model, "predict", o.init_model)
pred = utils.load_predicted(o, group_by="subset")

c = [v["z"]["joint"][:, :o.dim_c] for v in pred.values()]
subset_num = 28
c_ref = np.concatenate(c[:subset_num], axis=0)
c_query = np.concatenate(c[subset_num:], axis=0)
c_all = np.concatenate([c_query, c_ref], axis=0)

  0%|          | 0/40 [00:00<?, ?it/s]

Loading predicted variables ...
Loading subset 0: z, joint
Loading subset 1: z, joint
Loading subset 2: z, joint


  0%|          | 0/29 [00:00<?, ?it/s]

Loading subset 3: z, joint
Loading subset 4: z, joint


 93%|█████████▎| 25/27 [00:00<00:00, 247.59it/s]

...

  0%|          | 0/21 [00:00<?, ?it/s]

Loading subset 29: z, joint
Loading subset 30: z, joint
Loading subset 31: z, joint


100%|██████████| 15/15 [00:00<00:00, 219.98it/s]


Loading subset 32: z, joint
Loading subset 33: z, joint
Converting to numpy ...
Converting subset 0: s, joint
Converting subset 0: z, joint
Converting subset 1: s, joint
Converting subset 1: z, joint
Converting subset 2: s, joint
Converting subset 2: z, joint
Converting subset 3: s, joint
Converting subset 3: z, joint


Converting subset 4: s, joint
Converting subset 4: z, joint
Converting subset 5: s, joint
Converting subset 5: z, joint
Converting subset 6: s, joint
Converting subset 6: z, joint
Converting subset 7: s, joint
Converting subset 7: z, joint
Converting subset 8: s, joint
Converting subset 8: z, joint
Converting subset 9: s, joint
Converting subset 9: z, joint
Converting subset 10: s, joint
Converting subset 10: z, joint
Converting subset 11: s, joint
Converting subset 11: z, joint
Converting subset 12: s, joint
Converting subset 12: z, joint
Converting subset 13: s, joint
Converting subset 13: z, joint
Converting subset 14: s, joint
Converting subset 14: z, joint
Converting subset 15: s, joint
Converting subset 15: z, joint
Converting subset 16: s, joint
Converting subset 16: z, joint
Converting subset 17: s, joint
Converting subset 17: z, joint
Converting subset 18: s, joint
Converting subset 18: z, joint
Converting subset 19: s, joint
Converting subset 19: z, joint
Converting subset 20

In [4]:
label_atlas = np.array(utils.transpose_list(utils.load_csv(pj("analysis", "atlas_label", "atlas_new.csv")))[1][1:])
label_atlas = np.concatenate([label_atlas[:114343], label_atlas[132554:185518], label_atlas[196042:], label_atlas[185518:196042], label_atlas[114343:132554]], axis=0)
label_gt = label_atlas[:len(c_ref)]
label_query = label_atlas[len(c_ref):]

In [5]:
knn = KNeighborsClassifier(n_neighbors=20, weights='distance')
knn.fit(c_ref, label_gt)
label_pred = knn.predict(c_query)
label_query_keys = np.unique(label_query)
label_pred_keys = np.unique(label_pred)
results = {}
results["confusion"] = confusion_matrix(label_query, label_pred, labels=label_query_keys)
results["f1"] = f1_score(label_query, label_pred, average='micro')
print(20, " f1: ", results["f1"])

20  f1:  0.8588132938924656


In [6]:
result_dir = pj("result", "comparison", o.task, o.method, o.experiment, o.model, o.init_model)
utils.mkdirs(result_dir, remove_old=False)
utils.save_list_to_csv([list(line) for line in list(label_pred.reshape(-1, 1))], pj(result_dir, "label_transferred.csv"))
utils.save_list_to_csv([list(line) for line in list(label_query.reshape(-1, 1))], pj(result_dir, "label_gt.csv"))